In [81]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from sklearn import preprocessing, tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import pydotplus 
from sklearn import metrics
from IPython.display import Image 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

In [82]:
dff=pd.read_json('../01_Preprocessing/Second.json').sort_index()

In [83]:
dff.head(2)

,country,countryCode,degree,degreeCode,eng,field,fieldG_0,fieldG_1,fieldG_10,fieldG_11,...,greQ,greV,greW,highBU,highMU,paper,rejUni,targetUni,uniRank,year
2,united kingdom,4,Doctorate,1,8.0,petroleum engineering,False,False,False,False,...,NaN,NaN,NaN,False,False,7,n/a,heriot-watt,692.0,2007
3,united states,0,Doctorate,1,8.0,civil,False,False,False,False,...,159.0,143.0,3.0,True,False,11,"UC Berkeley, Georgia Tech",northwestern,20.0,2010


# RandomForest (For MS)

In [84]:
from sklearn.ensemble import RandomForestClassifier

In [85]:
dff.columns

Index([u'country', u'countryCode', u'degree', u'degreeCode', u'eng', u'field',
       u'fieldG_0', u'fieldG_1', u'fieldG_10', u'fieldG_11', u'fieldG_12',
       u'fieldG_13', u'fieldG_14', u'fieldG_15', u'fieldG_16', u'fieldG_17',
       u'fieldG_18', u'fieldG_2', u'fieldG_3', u'fieldG_4', u'fieldG_5',
       u'fieldG_6', u'fieldG_7', u'fieldG_8', u'fieldG_9', u'fieldGroup',
       u'fund', u'fundValue', u'gpaB', u'gpaM', u'gre', u'greQ', u'greV',
       u'greW', u'highBU', u'highMU', u'paper', u'rejUni', u'targetUni',
       u'uniRank', u'year'],
      dtype='object')

In [86]:
col=['countryCode', 'eng',
       'fieldG_0', 'fieldG_1', 'fieldG_10', 'fieldG_11', 'fieldG_12',
       'fieldG_13', 'fieldG_14', 'fieldG_15', 'fieldG_16', 'fieldG_17',
       'fieldG_18', 'fieldG_2', 'fieldG_3', 'fieldG_4', 'fieldG_5',
       'fieldG_6', 'fieldG_7', 'fieldG_8', 'fieldG_9',
       'gpaB', 'gre', 'greQ', 'greV', 'greW', 
       'highBU', 'paper', 'uniRank']

In [87]:
dff=dff[dff.degreeCode==0]
df=dff[col]

In [88]:
df.uniRank.fillna(300,inplace=True)
df.greQ.fillna(0,inplace=True)
df.greW.fillna(0,inplace=True)
df.greV.fillna(0,inplace=True)

In [89]:
df.head()

,countryCode,eng,fieldG_0,fieldG_1,fieldG_10,fieldG_11,fieldG_12,fieldG_13,fieldG_14,fieldG_15,...,fieldG_8,fieldG_9,gpaB,gre,greQ,greV,greW,highBU,paper,uniRank
6,2,7.0,True,False,False,False,False,False,False,False,...,False,False,3.0,True,166.0,138.0,3.0,True,0,443.0
7,3,8.0,False,False,True,False,False,False,False,False,...,False,False,2.5,False,0.0,0.0,0.0,False,0,300.0
8,1,6.5,False,False,True,False,False,False,False,False,...,False,False,3.0,False,0.0,0.0,0.0,True,0,318.0
9,0,8.0,True,False,False,False,False,False,False,False,...,False,False,3.5,False,0.0,0.0,0.0,True,1,21.0
12,0,7.5,True,False,False,False,False,False,False,False,...,False,False,3.5,True,155.0,146.0,3.5,False,1,300.0


In [91]:
xtr,xte,ytr,yte=train_test_split(df,dff.fund,test_size=0.2 ,random_state=99)

In [92]:
def build3(xtr, ytr,xte,yte,rocEnable):
    clf = RandomForestClassifier(n_estimators=10,random_state=99)
    clf.fit(xtr, ytr)
    ypred = clf.predict(xte)
    if rocEnable==False:
        acc=accuracy_score(yte,ypred)*100
    else:
        fpr, tpr, thresholds = metrics.roc_curve(yte,ypred)
        acc=metrics.auc(fpr, tpr)*100
    conf=confusion_matrix(yte,ypred)
    return (clf,acc,conf)
def draw3(clf):
    dot_data = tree.export_graphviz(clf, out_file=None, feature_names=col,  max_depth=3, rounded=True)  
    graph = pydotplus.graph_from_dot_data(dot_data)  
    return Image(graph.create_png())  

In [94]:
build3(xtr, ytr,xte,yte,True)

(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=10, n_jobs=1, oob_score=False, random_state=99,
             verbose=0, warm_start=False),
 63.120389526257505,
 array([[373,  36],
        [ 76,  41]]))

In [95]:
build3(xtr, ytr,xte,yte,False)

(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=10, n_jobs=1, oob_score=False, random_state=99,
             verbose=0, warm_start=False),
 78.707224334600753,
 array([[373,  36],
        [ 76,  41]]))